In [1]:
from jinja2 import Environment, FileSystemLoader, select_autoescape
import os
import shutil
import datetime

In [8]:
template_name = "single_run"

env = Environment(
    block_start_string = '\BLOCK{',
    block_end_string = '}',
    variable_start_string = '\VAR{',
    variable_end_string = '}',
    comment_start_string = '\#{',
    comment_end_string = '}',
    line_statement_prefix = '%%',
    line_comment_prefix = '%#',
    trim_blocks = True,
    autoescape = False,
    loader = FileSystemLoader(os.path.abspath("."))
)

# Nuke Build
if os.path.exists("build"):
    shutil.rmtree("build")
os.mkdir("build")


# Render Header, Commands, Packages
options = {
    "date": datetime.date.today().strftime('%B %-d, %Y'),
}

for file_name in ["header", "commands", "packages"]:
    template = env.get_template("{}.jinja.tex".format(file_name))
    renderer_template = template.render(**options)
    with open("build/{}.tex".format(file_name), "w") as f:
        f.write(renderer_template)
        
# Render Main Template
template_path = "templates/{}/{}.jinja.tex".format(template_name, template_name)
if not os.path.exists(template_path):
    raise ValueError('The template "{}" does not exist'.format(template_path))

_src = "templates/{}/".format(template_name)
_dest = "build/"
_src_files = os.listdir(_src)
for _file_name in _src_files:
    if not _file_name in [".DS_Store", "{}.jinja.tex".format(template_name)]:
        _full_file_name = os.path.join(_src, _file_name)
        if (os.path.isfile(_full_file_name)):
            shutil.copy(_full_file_name, _dest)
        
template = env.get_template(template_path)
renderer_template = template.render(**options)
with open("build/{}.tex".format(template_name), "w") as f:
    f.write(renderer_template)

In [ ]:
?shutil.copytree

In [ ]:
""" Recursive Render Function """
def render(indir, outdir, env, static, sizes):
	for direntry in os.scandir(indir):
		if direntry.is_dir():
			os.mkdir(os.path.join(outdir, direntry.name))
			render(
				indir=os.path.join(indir, direntry.name),
				outdir=os.path.join(outdir, direntry.name),
				env=env, static=static, sizes=sizes)
		else:
			if (not direntry.name.startswith('_')) and (direntry.name.endswith('.html')):
				print("rendering: {}/{}".format(indir, direntry.name))
				if indir.startswith('templates/'):
					_l = (list(os.path.split(indir))[1:])
					_l.append(direntry.name)
					template = env.get_template(os.path.join(*_l))
				else:
					template = env.get_template(direntry.name)

				with open(os.path.join(outdir, direntry.name), 'w') as file:
					file.write(template.render(static=static, size=sizes))

In [ ]:
#!/usr/bin/env python3

import os
import shutil
from jinja2 import Environment, FileSystemLoader, select_autoescape
from hashlib import md5
from hurry.filesize import size as hurry_size
import sys

"""Add md5 hash to all static files"""
def static_versioning():
	# preprocess static files
	if os.path.exists('static_tmp'):
		shutil.rmtree('static_tmp')
	shutil.copytree("static", "static_tmp")

	sys.stdout.write("Compressing images ...")
	sys.stdout.flush()
	os.system('find static_tmp -type f -name "*.jpg" -exec convert {} -sampling-factor 4:2:0 -define jpeg:extent=200kb -strip -quality 85 -interlace JPEG -colorspace sRGB {} \;')
	sys.stdout.write("\n")
	sys.stdout.flush()

	# fill site/static
	os.mkdir("site/static")

	static = dict()
	sizes = dict()
	exts = [".jpg", ".jepg", ".pdf", ".png", ".css", ".ico", ".svg"]
	for direntry in os.scandir("static_tmp"):
		if direntry.is_file():
			filename, fileext = os.path.splitext(direntry.name)

			if fileext in exts:
				with open(direntry.path, 'rb') as file:
					md5_hash = md5(file.read()).hexdigest()

				new_name = "{}_md5={}{}".format(filename, md5_hash, fileext)
				static[direntry.name] = "/{}".format(os.path.join("static", new_name))
				statinfo = direntry.stat()
				sizes[direntry.name] = hurry_size(statinfo.st_size)
				shutil.copyfile(
					os.path.join("static_tmp",direntry.name),
					os.path.join("site", "static", new_name))

	# clean
	shutil.rmtree('static_tmp')

	return static, sizes

""" Recursive Render Function """
def render(indir, outdir, env, static, sizes):
	for direntry in os.scandir(indir):
		if direntry.is_dir():
			os.mkdir(os.path.join(outdir, direntry.name))
			render(
				indir=os.path.join(indir, direntry.name),
				outdir=os.path.join(outdir, direntry.name),
				env=env, static=static, sizes=sizes)
		else:
			if (not direntry.name.startswith('_')) and (direntry.name.endswith('.html')):
				print("rendering: {}/{}".format(indir, direntry.name))
				if indir.startswith('templates/'):
					_l = (list(os.path.split(indir))[1:])
					_l.append(direntry.name)
					template = env.get_template(os.path.join(*_l))
				else:
					template = env.get_template(direntry.name)

				with open(os.path.join(outdir, direntry.name), 'w') as file:
					file.write(template.render(static=static, size=sizes))


if os.path.exists('site'):
	shutil.rmtree('site')

os.mkdir('site')

# static versioning, i.e. appending md5 hash to filename
static, sizes=static_versioning()

# Render Templates
env = Environment(
	loader=FileSystemLoader('templates'),
	autoescape=select_autoescape(['html', 'xml']),
)

render(indir='templates', outdir='site', env=env, static=static, sizes=sizes)